In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install pydantic
!pip install python-dotenv

In [1]:
# =========================
# BLOCK 0 — ENVIRONMENT SETUP
# =========================

import os
import sys
import sqlite3
from dotenv import load_dotenv
import importlib.metadata

# Project root
project_root = os.path.abspath("..")
sys.path.append(project_root)

# Load environment variables
load_dotenv(os.path.join(project_root, ".env"))

# =========================
# SPOTIFY SETUP
# =========================

import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = (
    "playlist-read-private "
    "playlist-modify-private "
    "playlist-modify-public "
    "user-library-read "
    "user-read-recently-played"
)

cache_path = os.path.join(project_root, ".spotify_cache")

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        scope=scope,
        cache_path=cache_path
    )
)

print("Spotify authenticated.")
print("Current user:", sp.current_user()["id"])

# =========================
# DATABASE SETUP
# =========================

from core.database import get_connection, create_tables

db_path = os.path.join(project_root, "music_agent.db")
conn = get_connection(db_path)
create_tables(conn)

print("Database connected.")
print("SQLite version:", sqlite3.sqlite_version)

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables:", cursor.fetchall())

print("Spotipy version:", importlib.metadata.version("spotipy"))

# =========================
# LANGCHAIN SETUP
# =========================

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

print("LangChain LLM ready.")


Spotify authenticated.
Current user: 21ogollmd4ljapwluvwxe3stq
Database connected.
SQLite version: 3.45.3
Tables: [('tracks',), ('artists',), ('albums',), ('track_artists',), ('track_albums',), ('playlists',), ('playlist_tracks',), ('saved_albums',), ('track_audio_features',), ('track_metrics',), ('playlist_metrics',), ('play_history',)]
Spotipy version: 2.25.2
LangChain LLM ready.


In [3]:
from core.ingestion import sync_new_tracks
from core.database import get_latest_added_at

new_tracks = sync_new_tracks(sp, conn, get_latest_added_at)
print("New tracks inserted:", new_tracks)

cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM tracks;")
print("Total tracks:", cursor.fetchone()[0])


New tracks inserted: 721
Total tracks: 721


In [ ]:
from core.langchain_tools import build_rename_tool
from core.agent import build_agent

# Build tool
rename_tool = build_rename_tool(sp)

# Build agent
agent = build_agent([rename_tool])

# Test input
rresponse = agent.invoke({
    "messages": [
        {"role": "user", "content": "Rename playlist Drake Songs to Drake Songs 2026"}
    ]
})

print(response)


In [ ]:
response = agent.invoke({
    "messages": [
        {"role": "user", "content": "Rename playlist Drake Songs 2026 to Drake Songs 2027"}
    ]
})

print(response)


In [5]:
from core.playlists import get_playlist_id_by_name

playlist_name = "Drake Songs 2027"  # usa una playlist que realmente exista

playlist_id = get_playlist_id_by_name(sp, playlist_name)

print("Playlist ID:", playlist_id)


Playlist ID: 5hCRnQvrICA4O6kIP2X1fF


In [7]:
def create_playlist(sp, name: str, public: bool = False):
    """
    Create a new playlist and return its ID.
    """
    playlist = sp._post(
        "me/playlists",
        payload={
            "name": name,
            "public": public
        }
    )

    return playlist["id"]


In [8]:
create_playlist

new_playlist_id = create_playlist(sp, "Test Architecture Playlist", public=False)

print("New playlist ID:", new_playlist_id)


New playlist ID: 51yllYSMvcGIo3TrxsfVxI


In [9]:
from core.playlists import update_playlist_details

update_playlist_details(sp, new_playlist_id, name="Test Architecture Playlist Updated")

print("Rename executed.")


Rename executed.


In [11]:
def add_tracks_to_playlist(sp, playlist_id: str, track_ids: list):
    """
    Add track IDs to a playlist using official /items endpoint.
    """
    track_uris = [f"spotify:track:{tid}" for tid in track_ids]

    for i in range(0, len(track_uris), 100):
        sp._post(
            f"playlists/{playlist_id}/items",
            payload={
                "uris": track_uris[i:i+100]
            }
        )


In [12]:
from core.repository import get_recent_tracks


# Get 3 recent tracks from SQLite
track_ids = get_recent_tracks(conn, limit=3)

print("Track IDs:", track_ids)

# Add them to the new playlist
add_tracks_to_playlist(sp, new_playlist_id, track_ids)

print("Tracks added.")


Track IDs: ['6MXXY2eiWkpDCezVCc0cMH', '7rbECVPkY5UODxoOUVKZnA', '0j2T0R9dR9qdJYsB7ciXhf']
Tracks added.


In [13]:
from core.playlists import remove_tracks_from_playlist

# Remove the same tracks we just added
remove_tracks_from_playlist(sp, new_playlist_id, track_ids)

print("Tracks removed successfully.")


RuntimeError: Failed to remove tracks. Status: 400, Response: {"error": {"status": 400, "message": "No uris provided" } }